<a href="https://colab.research.google.com/github/Parkseojin2001/Baekjoon/blob/main/chapter11_part01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 11.2 텍스트 데이터 준비

### 11.2.4 TextVectorization 층 사용하기

In [1]:
import string

In [2]:
class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

In [3]:
vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]

In [4]:
vectorizer.make_vocabulary(dataset)

In [5]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)

In [6]:
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [7]:
decoded_sentence = vectorizer.decode(encoded_sentence)

In [8]:
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [9]:
# TextVectorization 층 사용

from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    output_mode="int",    # 정수 인덱스로 인코딩된 단어 시퀀스를 반환하도록 설정
)

In [10]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor)  # 문자열을 소문자로 변경
  return tf.strings.regex_replace(  # 구두점 문자를 빈 문자열로 변경
      lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
  return tf.strings.split(string_tensor)  # 공백을 기준으로 문자열 나누기

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [11]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

In [12]:
# 어휘 사전 출력하기
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [13]:
# 예시 문장 인코딩
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [14]:
# 예시 문장 디코딩
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


## 11.3 단어 그룹을 표현하는 두 가지 방법: 집합과 시퀀스

### IMDB 영화 리뷰 데이터 준비하기

In [15]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9195k      0  0:00:08  0:00:08 --:--:-- 12.3M


In [16]:
!rm -r aclImdb/train/unsup

In [17]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [18]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)    # 코드를 여러 번 실행해도 동일한 검증 세트를 생성
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]  # 훈련 파일 중 20%를 검증 세트로 이용
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)   # 파일 위치 이동

In [19]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [20]:
# 첫 번째 배치의 크기와 dtype 출력하기

for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Everything I remember about it was excellent... great cast with Sam Waterston & George Innes (before he became more familiar to US audiences).... excellent scripts as only the English can do - Edwardian Sherlock Holmes/Lord Peter Wimsey/Albert Campion type mysteries, but with a Jules Verne twist. Sort of like MacGyver would have been had it been in England 80 years earlier... right at the beginning of the scientific/technological revolution of the 20th century.<br /><br />I've often wondered if the creators of MacGyver saw these shows. MacGyver first aired about 3 years later.<br /><br />I still have 1 episode on a much deteriorated tape.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


### 11.3.2 단어를 집합으로 처리하기: BoW 방식

#### 이진 인코딩을 사용한 유니그램

In [21]:
# TextVectorization 층으로 데이터 전처리하기

text_vectorization = TextVectorization(
    max_tokens=20000,   # 가장 많이 등장하는 2만 개 단어로 어휘 사전 제한
    output_mode="multi_hot",  # 멀티-핫 이진 벡터로 출력 토큰 인코딩
)
text_only_train_ds = train_ds.map(lambda x, y: x)   # 원시 텍스트 입력만 반환
text_vectorization.adapt(text_only_train_ds) # adapt() 메서드로 데이터셋의 어휘사전 인덱싱
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)   # 훈련, 검증, 테스트셋 전처리

In [22]:
# 유니그램 데이터셋의 출력 확인하기

for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape:", inputs.shape)  # 입력은 20,000차원 벡터 배치
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])  # 0과 1로만 구성
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [23]:
# 모델 생성 유틸리티

from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation="sigmoid")(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics=["accuracy"])
  return model

In [24]:
# 이진 유니그램 모델 훈련하고 테스트하기

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.h5",
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),  # cache() 메서드를 호출하여 메모리에 캐싱

          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.h5")
print(f"테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 10s 8ms/step - loss: 0.4044 - accuracy: 0.8289 - val_loss: 0.2942 - val_accuracy: 0.8862


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2735 - accuracy: 0.8986 - val_loss: 0.2914 - val_accuracy: 0.8866
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2481 - accuracy: 0.9118 - val_loss: 0.2947 - val_accuracy: 0.8902
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2250 - accuracy: 0.9237 - val_loss: 0.3157 - val_accuracy: 0.8864
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2161 - accuracy: 0.9276 - val_loss: 0.3245 - val_accuracy: 0.8890
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2166 - accuracy: 0.9301 - val_loss: 0.3409 - val_accuracy: 0.8862
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2055 - accuracy: 0.9345 - val_loss: 0.3502 - val_accuracy: 0.8880
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1977 - accuracy: 0.9363 - val_loss: 0.3705 - val_accuracy: 0.8844
Epoch 

#### 이진 인코딩을 사용한 바이그램

In [25]:
# 바이그램을 반환하는 TextVectorization 층 만들기

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

In [26]:
# 이진 바이그램 모델 훈련하고 테스트하기

text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3710 - accuracy: 0.8461 - val_loss: 0.2701 - val_accuracy: 0.8952
Epoch 2/10
62

#### TF-IDF 인코딩을 사용한 바이그램

In [27]:
# 토큰 카운트를 반환하는 TextVectorization 층

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

In [28]:
# TF-IDF 가중치가 적용된 출력을 반환하는 TextVectorization 층

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

In [29]:
# TF-IDF 바이그램 모델 훈련하고 테스트하기

text_vectorization.adapt(text_only_train_ds) # 어휘 사전과 TF-IDF 가중치 학습

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.h5",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.h5")
print(f"테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.4898 - accuracy: 0.7847 - val_loss: 0.2834 - val_accuracy: 0.8976
Epoch 2/10
6